In [2]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.3.0-cp310-cp310-macosx_12_0_arm64.whl size=417082 sha256=8ea3db9908f88e5cac7ff219f68a95e14d28d2f0332f91a087e5d3e5306eb3dc
  Stored in directory: /Users/s_gre1/Library/Caches/pip/wheels/60/e1/7f/881b5af199acf453d55d49d38e227d291fe5b562099ac29a68
Successfully built scikit-learn-extra
Note: you may need to restart the kernel to use updated packages.


### Helper Functions: Algorithms & Clustering

**Algorithms**

TO DO: add a "mode" argument to each algorithm that, if mode = 1 the cluster labelling is output and if mode = 0 the silhouette coefficient is output.

In [1]:
# Code for KMeans

import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from sklearn.metrics import silhouette_score

def kmeans_clustering(samples, mode, n_clusters=2, max_iter=300):
    """
    Perform KMeans clustering on the input samples
    
    Parameters:
        samples: array-like, shape (n_samples, n_features)
        n_clusters: int, number of clusters (default=2)
        max_iter: int, maximum iterations (default=300)
    
    Returns:
        silhouette_coef: silhouette coefficient score
    """
    k_means = KMeans(n_clusters=n_clusters, max_iter=max_iter)
    k_means.fit(samples)

    # Get cluster labels
    labels = k_means.labels_

    # If mode=1, return the cluster labels
    if mode == 1:
        return labels
    
    # Otherwise, calculate and return the silhouette score
    try:
        silhouette_coef = silhouette_score(samples, k_means.labels_, metric='euclidean')
    except ValueError:
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    return silhouette_coef

In [ ]:
# EM Clustering Code

from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def em_clustering(selected_features, mode, n_clusters=2):
    """
    Perform EM Clustering on selected features and return silhouette score.
        
    Returns:
    --------
    float
        Silhouette score of the clustering (-1 if clustering fails)
    """
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the EM model
    em_model = GaussianMixture(
        n_components=n_clusters,
        random_state=0, #THOUGHTS: We can improve this later to have an array of seeds to select from to observe variations
        n_init=10  # Multiple initializations to avoid local optima
    )
    
    try:
        # Fit the model and get cluster assignments
        em_model.fit(X_scaled)
        labels = em_model.predict(X_scaled)

        # If mode=1, return the cluster labels
        if mode == 1:
            return labels
        
        # Calculate silhouette score
        silhouette_coef = silhouette_score(X_scaled, labels)
    except Exception as e:
        #print(f"Clustering failed: {str(e)}")
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    
    return silhouette_coef


In [ ]:
# DBSCAN Detection method: 
# I put 'optimization part' in 'DBSCAN_Optimization_Code.ipynb' file. 
# We can use optimization after initial run to do a comparison and analysis in our paper to show improvements.

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def dbscan_clustering(selected_features, mode, eps=0.5, min_samples=5):
    """
    Perform DBSCAN clustering on selected features
    
    Parameters:
    selected_features : pandas DataFrame
        The features selected for clustering
    eps : float
        The maximum distance between two samples for them to be considered neighbors
    min_samples : int
        The number of samples in a neighborhood for a point to be considered a core point
        
    Returns:
    float : silhouette coefficient
    dict : additional clustering information
    """
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize and fit DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)
    
    # Get number of clusters (excluding noise points which are labeled -1, K Medoids does not have noise points)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # If mode=1, return the cluster labels
    if mode == 1:
        return labels
    
    # Calculate silhouette score if more than one cluster and no noise points
    if n_clusters > 1 and -1 not in labels:
        silhouette_coef = silhouette_score(X_scaled, labels)
    else:
        silhouette_coef = 0  # Assign lowest score if clustering fails

    
    # NOTE: -- Uncomment when we analyze and optimize ---- Additional clustering information
    # info = {
    #     'n_clusters': n_clusters,
    #     'n_noise': list(labels).count(-1),
    #     'labels': labels,
    #     'cluster_sizes': pd.Series(labels).value_counts().to_dict()
    # }
    
    return silhouette_coef


In [2]:
# Code for K Medoids
import numpy as np
import pandas as pd

from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def kmedoids_clustering(selected_features, mode, n_clusters=2, max_iter=1000):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the K-Medoids model
    kmedoids = KMedoids(n_clusters=n_clusters, method='pam', max_iter=max_iter, random_state=0)
    labels = kmedoids.fit_predict(X_scaled)

    # If mode=1, return the cluster labels
    if mode == 1:
        return labels

    # Otherwise, calculate and return the silhouette score
    try:
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assigning lowest score if clustering fails

    return silhouette_coef

In [3]:
# Codes for Mean Shift
import numpy as np
import pandas as pd

from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def meanshift_clustering(selected_features, mode, bandwidth=None):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the Mean Shift model
    meanshift = MeanShift(bandwidth=bandwidth)
    labels = meanshift.fit_predict(X_scaled)

    # Check and print the number of clusters determined 
    n_clusters = len(np.unique(labels))
    print(f"Number of clusters found: {n_clusters}")

    # If mode=1, return the cluster labels
    if mode == 1:
        return labels

    # Otherwise, calculate and return the silhouette score
    try:
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assign lowest score if clustering fails
    
    return silhouette_coef

**Clustering & Output**

In [ ]:
''' Takes in the state (feature configuration) and action (algorithm) that
produced the max value in the Q-Matrix to produce the final cluster'''
def get_cluster(state, action):
  

### Main Code

In [113]:
import pandas as pd
import numpy as np

FEATURES = {0: 'avg_bytes_sent', 1: 'avg_bytes_received', 2: 'avg_packets_transferred', 
  3: 'avg_flow_duration', 4: 'recent_tcp_flags', 5: 'recent_protocol', 6: 'avg_cpu_usage', 
  7: 'avg_memory_usage', 8: 'avg_disk_usage', 9: 'avg_uptime'}

data = pd.read_csv("joined_quantitative_data.csv")

ALGORITHMS = {0: 'K-Means', 1: 'Mean Shift', 2: 'K-Mediods', 3: 'EM Clustering', 4: 'DBSCAN Clustering'}
NUM_ALG = len(ALGORITHMS)
original_features = data.iloc[:, 2:]
ips = data['source_ip']
#print(original_features.columns)
#print(ips.head(10))

def algorithm_prep(state, action, mode):
  # convert state to binary
  state_bin = bin(state)
  #print(state_bin)
  state_bin_arr = np.array([b for b in state_bin[2:]])
  # pad with zeros
  diff = 10 - len(state_bin_arr)
  padded_arr = np.insert(state_bin_arr, 0, ['0' for i in range(diff)])
  #(padded_arr)
  # identify which indexes are 1
  idx = (np.where(padded_arr == '1')[0]).tolist()
  #print(idx)
  # select feature headings
  selected_features = original_features.iloc[:,idx]
  #print(selected_features.head(10))
  # select algorithm
  # algo = action
  # prep correct data - done
  
  # call algorithm function
  out = None
  #print('algorithm:',ALGORITHMS[action])

  # if mode = 0, output is the silhouette coefficient
  # if mode = 1, output is the cluster labelling
  match action:
    case 0:
      #print('algorithm:',ALGORITHMS[action])
      out = kmeans_clustering(selected_features, mode)
    case 1: 
      #print('algorithm:',ALGORITHMS[action])
      out = meanshift_clustering(selected_features, mode)
    case 2:
      #print('algorithm:',ALGORITHMS[action])
      out = kmedoids_clustering(selected_features, mode)
    case 3: 
      out = em_clustering(selected_features, mode)
    case 4:
      out = dbscan_clustering(selected_features, mode)
  # return silhouette from algorithm function
  return out

In [119]:
algorithm_prep(1, 0,0)

np.float64(0.6307170631786834)

In [90]:
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2018 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
# 1024 configurations of the 10 features --> 2^10
# 5 algorithms
R = ql.matrix(ql.zeros([1024,5]))

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([1024,5]))

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
# 1) TO-DO: decide if starting state is random or a specific state
agent_s_state = 1

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    # 2) DONE: we should check Q, not R because R is never modified
    current_state_row = Q[state,]
    # 3) DONE: this should pick valid actions based on what we have not visited
    possible_act = ql.where(current_state_row == 0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1)[0])
    if(sum(PossibleAction)<=0):
        next_action = int(np.random.choice(NUM_ALG+1,1)[0])
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

# A version of Bellman's equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1)[0])
    else:
        Max_State = int(Max_State[0])

    # 5) DONE: we think this is a typo and action/Max_State should be switched. 
    # MaxValue = Q[action, Max_State]
    MaxValue = Q[Max_State, action]

    # 6) DONE: call function to run ML algorithm using the value of action. this will
    # run the algorithm using the features from current_state, create clusters,
    # and calculate the silhouette value.
    silhouette_co = algorithm_prep(current_state, action, 0) 
    
    # Bellman's MDP based Q function
    # 7) DONE: instead of getting a value from R, we add the silhouette value to gamma * MaxValue
    # Q[current_state, action] = R[current_state, action] + gamma * MaxValue
    Q[current_state, action] = silhouette_co + gamma * MaxValue


# Rewarding Q matrix
reward(agent_s_state,action,gamma)


# Leraning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(6000):
    # select a random new state (configuration of features)
    current_state = ql.random.randint(1, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

# DONE: get maximum value from Q-Learning Matrix
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("max value located at",max_location)
max_config = max_location[0][0]
max_algorithm = ALGORITHMS[max_location[1][0]]
print(f"Using algorithm {max_algorithm} and feature configuration {max_config}, max value is:",normed_Q[278,0])

# TO-DO: get final cluster labels
cluster_labels = get_cluster(max_config, max_algorithm)

# TO-DO: match data in clusters to IP addresses


# TO-DO: return what IPs are likely anomalous

/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/homebrew/lib/python3.10/site-packages/s

Clustering failed: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceW

Clustering failed: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning

Clustering failed: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceW

Q  :
[[ 0.          0.          0.          0.          0.        ]
 [ 1.12714607  0.52921146  0.63071706  0.63071706 -1.        ]
 [ 0.55761235 -1.8         0.55761235  0.55929848 -1.        ]
 ...
 [ 1.4163     -0.74825001  0.13145184  0.79819204 -1.8       ]
 [ 1.7712977  -0.10722941  0.1199422   0.7984389  -1.        ]
 [ 1.87192697  0.665209    0.396487    0.78066054 -1.        ]]
Normed Q :
[[  0.           0.           0.           0.           0.        ]
 [ 59.46257281  27.91854209  33.27346864  33.27346864 -52.75498411]
 [ 29.41683065 -94.95897139  29.41683065  29.50578268 -52.75498411]
 ...
 [ 74.71688411 -39.47391749   6.93473992  42.10860817 -94.95897139]
 [ 93.44478178  -5.65688581   6.32754911  42.12163172 -52.75498411]
 [ 98.75347744  35.09309018  20.91666532  41.18373425 -52.75498411]]


In [101]:
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("max value located at",max_location)
max_config = max_location[0][0]
max_algorithm = ALGORITHMS[max_location[1][0]]
print(f"Using algorithm {max_algorithm} and feature configuration {max_config}, max value is:",normed_Q[278,0])



max value located at (array([278]), array([0]))
278
Using algorithm K-Means and feature configuration 278, max value is: 100.0


## Additional Reference Code

In [4]:
# -*- coding: utf-8 -*-
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2019 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
R = ql.matrix([ [0,0,0,0,1,0],
		            [0,0,0,1,0,1],
		            [0,0,100,1,0,0],
	             	[0,1,1,0,1,0],
		            [1,0,0,1,0,0],
		            [0,1,0,0,0,0] ])

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([6,6]))

"""##  The Learning rate or training penalty"""

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

"""## Initial State"""

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
agent_s_state = 5

"""## The random choice of the next state"""

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    current_state_row = R[state,]
    possible_act = ql.where(current_state_row >0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1))
    if(sum(PossibleAction)<=0):
        next_action = int(ql.random.choice(5,1))
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

"""## The Bellman Equation"""

# A version of the Bellman equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1))
    else:
        Max_State = int(Max_State)
    MaxValue = Q[action, Max_State]
    
    # The Bellman MDP based Q function
    Q[current_state, action] = R[current_state, action] + gamma * MaxValue

# Rewarding Q matrix
reward(agent_s_state,action,gamma)

"""## Running the training episodes randomly"""

# Learning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(50000):
    current_state = ql.random.randint(0, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

"""# Improving the program by introducing a decision-making process"""
nextc=-1
nextci=-1
conceptcode=["A","B","C","D","E","F"]
origin=int(input("index number origin(A=0,B=1,C=2,D=3,E=4,F=5): "))
print("Concept Path")
print("->",conceptcode[int(origin)])
for se in range(0,6):
    if(se==0):
        po=origin
    if(se>0):
        po=nextci
        #print("se:",se,"po:",po)
    for ci in range(0,6):
        maxc=Q[po,ci]
        #print(maxc,nextc)
        if(maxc>=nextc):
            nextc=maxc
            nextci=ci
            #print("next c",nextc)
    if(nextci==po):
        break;
    #print("present origin",po,"next c",nextci," ",nextc," ",conceptcode[int(nextci)])
    print("->",conceptcode[int(nextci)])


Q  :
[[  0.      0.      0.      0.    258.44    0.   ]
 [  0.      0.      0.    321.8     0.    207.752]
 [  0.      0.    500.    321.8     0.      0.   ]
 [  0.    258.44  401.      0.    258.44    0.   ]
 [207.752   0.      0.    321.8     0.      0.   ]
 [  0.    258.44    0.      0.      0.      0.   ]]
Normed Q :
[[  0.       0.       0.       0.      51.688    0.    ]
 [  0.       0.       0.      64.36     0.      41.5504]
 [  0.       0.     100.      64.36     0.       0.    ]
 [  0.      51.688   80.2      0.      51.688    0.    ]
 [ 41.5504   0.       0.      64.36     0.       0.    ]
 [  0.      51.688    0.       0.       0.       0.    ]]
Concept Path
-> A
-> E
-> D
-> C
